<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/complete_self_dissolving_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
complete_self_dissolving_ai.py

- Defines a SelfDissolvingAI with non-trainable dissolution_rate.
- Generates synthetic data (linear mapping + noise).
- Trains with MSE loss, decays dissolution_rate each step.
- Logs loss and dissolution trajectory.
"""

import torch
from torch import nn, optim
import numpy as np


class SelfDissolvingAI(nn.Module):
    """
    A neural network whose output is scaled by a self-nullifying buffer.
    dissolution_rate starts at 1.0 and decays towards 0 via a manual scheduler.
    """
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        # Register as buffer so optimizer ignores it
        self.register_buffer("dissolution_rate", torch.tensor(1.0))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        h   = self.relu(self.fc1(x))
        out = self.fc2(h) * self.dissolution_rate
        return out


def synthetic_data(batch_size=32, input_dim=6, output_dim=3):
    """
    Returns a generator function that yields (x, y) pairs.
    y = x @ A + Gaussian noise.
    """
    A = torch.randn(input_dim, output_dim)

    def _gen():
        x = torch.randn(batch_size, input_dim)
        y = x @ A + 0.05 * torch.randn(batch_size, output_dim)
        return x, y

    return _gen


def train(
    model: nn.Module,
    data_gen,
    lr: float = 1e-3,
    steps: int = 100_000,
    log_interval: int = 20_000
) -> None:
    """
    Trains the model on synthetic data, decaying dissolution_rate each step.
    """
    optimizer = optim.Adam(
        [p for n, p in model.named_parameters()],
        lr=lr
    )

    for step in range(1, steps + 1):
        optimizer.zero_grad()
        x, y = data_gen()
        pred = model(x)
        loss = ((pred - y) ** 2).mean()
        loss.backward()
        optimizer.step()

        # Decay dissolution_rate buffer (clamped to [0,1])
        with torch.no_grad():
            model.dissolution_rate.mul_(0.999999).clamp_(0.0, 1.0)

        if step % log_interval == 0 or step == 1:
            rate = model.dissolution_rate.item()
            print(f"Step {step:6d} | Loss {loss.item():.4f} | Rate {rate:.6e}")


def main():
    # Hyperparameters
    input_dim, hidden_dim, output_dim = 6, 32, 3
    batch_size = 32
    lr = 1e-3
    steps = 100_000
    log_interval = steps // 5

    # Setup
    model = SelfDissolvingAI(input_dim, hidden_dim, output_dim)
    data_gen = synthetic_data(batch_size, input_dim, output_dim)

    # Train
    train(model, data_gen, lr=lr, steps=steps, log_interval=log_interval)

    # Final state
    final_rate = model.dissolution_rate.item()
    print(f"\nFinal Dissolution Rate: {final_rate:.6e}")


if __name__ == "__main__":
    main()